## Train deep regression model

This script trains the deep regression model and its output is under "**/deepnn**" which will be generated automatically.

It will use the python files under "**/model_files**" which includes the model definiton and other fuctions such as data loading and generating figures.

First, let's import packages.

In [1]:
#------ import packages ------#
import torch
from model_files.util import *
from model_files.nn_model import *
from tqdm import tqdm
import pandas as pd
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from torchsample.modules import ModuleTrainer

#plt.style.use("seaborn")
#plt.style.use("ggplot")
#plt.style.use("bmh")
plt.style.use("seaborn-darkgrid")
#plt.style.use("seaborn-deep")
#plt.style.use("tableau-colorblind10")

The function **train_deepreg** will start to train our deep regression models. It will call functions defined in **/model_files**. So please remember to have the **/model_files** before you run the code.

In [2]:
#------ define the function for training------#
def train_deepreg(trn_file, val_file, tst_file, 
                  normalization_type, gpu_id, 
                  trn_batch_size, val_batch_size, tst_batch_size,
                  layer_num, input_dim, hidden_dim, output_dim,learning_rate,max_epoch,check_freq):
    
    # set up folders to have training results
    dir_name = 'deepnn_results/training_results/'
    dir_model_name = 'deepnn_results/training_models'
    dir_name_test = 'deepnn_results/test_results/'
    make_dir([dir_name, dir_model_name, dir_name_test])
    
    # set up device
    device = torch.device("cuda:{}".format(gpu_id) if torch.cuda.is_available() else "cpu")
    print(device)
    
    # load training set, validation set, and held-out (test) set
    scaler_save_file_name = os.path.join(dir_model_name, '{}_scaler.gz'.format(normalization_type))
    scaler = generate_scaler(trn_file, scaler_save_file_name, normalization_type)
    
    #--- process training data, val data, test
    trn_features, trn_targets = get_normalized_dataset(trn_file, scaler)
    val_features, val_targets = get_normalized_dataset(val_file, scaler)
    tst_features, tst_targets = get_normalized_dataset(tst_file, scaler)
    
    #--- put data into dataloader
    trn_num = len(trn_targets)
    val_num = len(val_targets)
    tst_num = len(tst_targets)
    trn_loader = prepare_data(trn_features, trn_targets, trn_batch_size, trn_num)
    val_loader = prepare_data(val_features, val_targets, val_batch_size, val_num)
    tst_loader = prepare_data(tst_features, tst_targets, tst_batch_size, tst_num)
    
    #--- create model
    deep_reg_Net = DeepReg(layer_num, input_dim, hidden_dim, output_dim)
    print(deep_reg_Net)
    
    deep_reg_Net.to(device)
    
    
    #--- optimizer
    optimizer = torch.optim.Adam(params=deep_reg_Net.parameters(), lr=learning_rate)
    
    #--- loss fuction
    l2_loss_func = L2_Func()
    
    
    #---- start to train our network
    total_trn_loss = []
    total_trn_RMSE = []
    total_val_RMSE = []
    total_epoch = []

    train_RMSE = []
    val_RMSE = []

    best_RMSE = float('inf')
    best_epoch = 0
    
    for epoch in range(max_epoch):
        print('')
        print('')
        print('###################### Start to Train NN model ##########################')
        deep_reg_Net.train()
        epoch_loss = []
        progress = tqdm(total=len(trn_loader), desc='epoch % 3d' % epoch)
        for step, (X_features, Y_targets, idx) in enumerate(trn_loader):
            # zero the parameter gradients
            optimizer.zero_grad()
            
        
            ################## Get Training & Traget Dataset ##################
            X_features = X_features.to(device).float()
            Y_targets = Y_targets.to(device).flatten().float()
            
            # forward + backward + optimize
            Y_prediction = deep_reg_Net(X_features).flatten().float()
            loss = l2_loss_func(Y_prediction, Y_targets)  # MSE loss
            loss = torch.sqrt(loss) #RMSE
            loss.backward()
            optimizer.step()
            
            #---finished update in one batch
            epoch_loss.append(loss.data.cpu().numpy())
            progress.set_postfix({'loss': loss.data.cpu().numpy()})
            progress.update()
        progress.close()
        total_trn_loss.append(np.mean(epoch_loss))  #---- finished one epoch
        
        
        #------ validation our model
        if epoch%check_freq==0:
            trn_rmse, gt_pred_dict = val_deepreg(deep_reg_Net, trn_loader, device, l2_loss_func)
            total_trn_RMSE.append(trn_rmse)
            
            val_rmse, gt_pred_dict = val_deepreg(deep_reg_Net, val_loader, device, l2_loss_func)
            total_val_RMSE.append(val_rmse)

            if best_RMSE > val_rmse:
                best_RMSE = val_rmse
                best_epoch = epoch
                ################ check and always save the best model we have
                model_file_name = os.path.join(dir_model_name, 'best_net_L{}_H{}.pt'.format(layer_num, hidden_dim))
                save_model(deep_reg_Net.eval(), model_file_name)
            figure_name = os.path.join(dir_name, 'train_val_mse_L{}_H{}.png'.format(layer_num, hidden_dim))
            display_RMSE(total_trn_RMSE, total_val_RMSE, check_freq, figure_name)
            figure_name = os.path.join(dir_name, 'train_loss_L{}_H{}.png'.format(layer_num, hidden_dim))
            display_train_loss(total_trn_loss, figure_name)
    
    ###### after traing, let's get the best and verify its performance on training/validation/test set again!
    # load the best net
    model_file_name = os.path.join(dir_model_name, 'best_net_L{}_H{}.pt'.format(layer_num, hidden_dim))
    best_Net = DeepReg(layer_num, input_dim, hidden_dim, output_dim)
    if torch.cuda.is_available():
        try:
            best_Net.load_state_dict(torch.load(model_file_name, map_location='cuda:{}'.format(gpu_id)))
            print('Loading Pretrained models 1 (GPU)!')
        except:
            best_Net = nn.DataParallel(best_Net)
            best_Net.load_state_dict(torch.load(model_file_name, map_location='cuda:{}'.format(gpu_id)))
            print('Loading Pretrained models 2 (GPU)!')
    else:
        device = torch.device("cuda:{}".format(gpu_id) if torch.cuda.is_available() else "cpu")
        best_Net.load_state_dict(torch.load(model_file_name, map_location=torch.device('cpu')))
        print('Loading Pretrained models on CPU!')
        
        
    # check its performance on training set
    trn_rmse, trn_gt_pred_dict = val_deepreg(best_Net, trn_loader, device, l2_loss_func)
    
    # check its performance on validation set
    val_rmse, val_gt_pred_dict = val_deepreg(best_Net, val_loader, device, l2_loss_func)
    
    # check its performance on test set
    tst_rmse, tst_gt_pred_dict = val_deepreg(best_Net, tst_loader, device, l2_loss_func)
    
    
    #----------- Finally, let's save our results---------------#
    trn_pred_file = os.path.join(dir_name_test, 'train_prediction_L{}_H{}.csv'.format(layer_num, hidden_dim))
    trn_df = pd.DataFrame.from_dict(trn_gt_pred_dict)
    trn_df.to_csv(trn_pred_file, index=False)
    
    val_pred_file = os.path.join(dir_name_test, 'validation_prediction_L{}_H{}.csv'.format(layer_num, hidden_dim))
    val_df = pd.DataFrame.from_dict(val_gt_pred_dict)
    val_df.to_csv(val_pred_file, index=False)
    
    tst_pred_file = os.path.join(dir_name_test, 'hos_test_prediction_L{}_H{}.csv'.format(layer_num, hidden_dim))
    tst_df = pd.DataFrame.from_dict(tst_gt_pred_dict)
    tst_df.to_csv(tst_pred_file, index=False)
    
    final_RMSE_dict = {'Type':['Train_RMSE', 'Validation_RMSE', 'Test_RMSE'],
                       'RMSE':[trn_rmse, val_rmse, tst_rmse]}
    
    final_RMSE_df = pd.DataFrame.from_dict(final_RMSE_dict)
    final_RMSE_file = os.path.join(dir_name_test, 'final_RMSE_L{}_H{}.csv'.format(layer_num, hidden_dim))
    final_RMSE_df.to_csv(final_RMSE_file, index=False)
    
    print('')
    print('')
    print('>>>Congrats! The DNN regression model has been trained and saved!')

The function **deep_reg_val** is a validation function. It will test how our deep regression model works on validation set.

In [3]:
#------ define function for validation------#
def val_deepreg(model, data_loader, device, l2_loss_func):
    model.eval()
    RMSE = []
    gt_pred_dict = {'Y_True':[], 'Y_Prediction':[]}
    with torch.no_grad():
        for step, (X_features, Y_targets, idx) in enumerate(data_loader):
            X_features = X_features.to(device).float()
            Y_targets = Y_targets.to(device).flatten().float()
            Y_prediction = model(X_features).flatten().float().detach()
            cur_mse = l2_loss_func(Y_prediction, Y_targets)
            cur_mse = torch.sqrt(cur_mse) #RMSE
            RMSE.append(cur_mse.item())
            ##### let's save our ground truth and predictions
            Y_truth_list = list(Y_targets.cpu().numpy().flatten())
            Y_prediction_list = list(Y_prediction.cpu().numpy().flatten())
            gt_pred_dict['Y_True'].extend(Y_truth_list)
            gt_pred_dict['Y_Prediction'].extend(Y_prediction_list)
    avg_rmse = np.mean(RMSE)
    return avg_rmse, gt_pred_dict

The function **deep_reg_test** is a test function. It will test how our deep regression model works on test set.

Below, we start to run our code.

In [4]:
if __name__ == '__main__':
    ################ Parameters Settings ######################
    trn_file = '../../data/dnn_dataset/training.csv'
    val_file = '../../data/dnn_dataset/validation.csv'
    tst_file = '../../data/dnn_dataset/hos.csv'
    normalization_type = 'StandardScaler'
    gpu_id = 0
    trn_batch_size = 128
    val_batch_size = 128
    tst_batch_size = 128
    layer_num = 10
    input_dim = 19
    hidden_dim = 32
    output_dim = 1
    learning_rate = 1e-3
    
    max_epoch = 500
    check_freq = 1
    
    
    ################ Start Training ######################
    train_deepreg(trn_file,
                  val_file, 
                  tst_file, 
                  normalization_type, 
                  gpu_id, 
                  trn_batch_size, 
                  val_batch_size, 
                  tst_batch_size,
                  layer_num, 
                  input_dim, 
                  hidden_dim, 
                  output_dim,
                  learning_rate,
                  max_epoch,
                  check_freq)

cpu
DeepReg(
  (deepreg_layers): ModuleList(
    (0): Linear(in_features=19, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=32, out_features=32, bias=True)
    (9): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=32, out_features=32, bias=True)
    (13): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negative_slope=0.01)
    (15): Dropout(p=0.5, inplace=False)
    (16): Linear(in_features=32, out_

epoch   0: 100%|██████████| 35/35 [00:00<00:00, 89.78it/s, loss=23.399767]


The trained model has been saved!


###################### Start to Train NN model ##########################


epoch   1: 100%|██████████| 35/35 [00:00<00:00, 76.60it/s, loss=23.864504]


The trained model has been saved!


###################### Start to Train NN model ##########################


...



###################### Start to Train NN model ##########################


epoch  498: 100%|██████████| 35/35 [00:00<00:00, 127.47it/s, loss=6.279712] 




###################### Start to Train NN model ##########################


epoch  499: 100%|██████████| 35/35 [00:00<00:00, 157.41it/s, loss=6.168156] 


Loading Pretrained models on CPU!


>>>Congrats! The DNN regression model has been trained and saved!
